## Export data to SpaceTX format

First create the subclasses of `FetchedTile` and `TileFetcher` required and then export the data into SpaceTX

In [30]:
%load_ext autoreload
%autoreload 2

import functools
import os
from typing import Mapping, Tuple, Union

import click
import numpy as np
from skimage.io import imread
from slicedimage import ImageFormat

from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

import pdb

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return imread(file_path)

class StarMapTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue]
    ) -> None:
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
#        print(np.shape(self.tile_data()))
        return {Axes.Y: 7962, Axes.X: 3356}
#         return {Axes.Y: 1000, Axes.X: 500}
#         return {Axes.Y: 1024, Axes.X: 1024}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    @staticmethod
    def crop(img) -> np.ndarray:
#         crp = img[2000:6000, 1000:1800]
        crp = img[3000:4000, 1300:1800]
#         crp = img[2950:3150, 1330:1730]
#         crp = img[1024:2048, 0:1024]
        return crp
    
    def tile_data(self) -> np.ndarray:
#        return self.crop(imread(self.file_path))
       return (imread(self.file_path))
    
    def __str__(self) -> str:
        return self.file_path
    
class StarMapTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
#         self.num_z = 11
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C{ch_label+1}_Z{zplane_padded}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
class StarMapDAPITileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C0_Z{zplane_padded}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
def format_data(input_dir, output_dir):
    
    primary_image_dimensions: Mapping[Axes, int] = {
        Axes.ROUND: 1,
        Axes.CH: 4,
        Axes.ZPLANE: 11,
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
        "nuclei": {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: 11
        }
    }
    
#     pdb.set_trace()
    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=StarMapTileFetcher(input_dir),
        aux_tile_fetcher={"nuclei": StarMapDAPITileFetcher(input_dir)},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    
    codebook = [
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "GFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "RFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "Cy5"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 3, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "iRFP"
        }
    ]
    Codebook.from_code_array(codebook).to_json("/home/nomi/Desktop/starfish/experiment/codebook.json")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
%load_ext autoreload
%autoreload 2

import functools
import os
from typing import Mapping, Tuple, Union

import click
import numpy as np
from skimage.io import imread
from slicedimage import ImageFormat

from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

import pdb

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return imread(file_path)

class StarMapTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue]
    ) -> None:
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
        print(np.shape(self.tile_data()))
#         return {Axes.Y: 7962, Axes.X: 3356}
#         return {Axes.Y: 1024, Axes.X: 1024}
        return {Axes.Y: 1000, Axes.X: 500}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    @staticmethod
    def crop(img) -> np.ndarray:
#         crp = img[2000:6000, 1000:1800]
        crp = img[3000:4000, 1300:1800]
#         crp = img[2950:3150, 1330:1730]
#        crp = img[1024:2048, 0:1024]
        return crp
    
    def tile_data(self) -> np.ndarray:
        return self.crop(imread(self.file_path))
#         return (imread(self.file_path))
    
    def __str__(self) -> str:
        return self.file_path
    
class StarMapTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
#         self.num_z = 11
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C2_Z{zplane_padded}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
class StarMapDAPITileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C{ch_label}_Z{zplane_padded}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
def format_data(input_dir, output_dir):
    
    primary_image_dimensions: Mapping[Axes, int] = {
        Axes.ROUND: 1,
        Axes.CH: 1,
        Axes.ZPLANE: 11,
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
        "nuclei": {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: 11
        }
    }
    
#     pdb.set_trace()
    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=StarMapTileFetcher(input_dir),
        aux_tile_fetcher={"nuclei": StarMapDAPITileFetcher(input_dir)},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    codebook = [
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "RFP"
        }
    ]
    Codebook.from_code_array(codebook).to_json("/home/nomi/Desktop/starfish/experiment/codebook.json")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
format_data("/home/nomi/Desktop/starfish/raw_data/2019-06-29_Justus_section3_round1_2x10_1",
           "/home/nomi/Desktop/starfish/experiment")

NameError: name 'format_data' is not defined

## Load the experiment and visualize the codebook
Possibly necessary to copy the codebook information from `codebook_backup.json`.

In [1]:
%load_ext autoreload
%autoreload 2
from starfish import Experiment
from six.moves import urllib
from slicedimage.backends import CachingBackend, DiskBackend, HttpBackend, SIZE_LIMIT

baseurl="/home/nomi/Desktop/starfish/experiment/experiment.json"
experiment = Experiment.from_json(baseurl)

In [2]:
experiment.codebook

<xarray.Codebook (target: 4, c: 4, r: 1)>
array([[[1],
        [0],
        [0],
        [0]],

       [[0],
        [1],
        [0],
        [0]],

       [[0],
        [0],
        [1],
        [0]],

       [[0],
        [0],
        [0],
        [1]]], dtype=uint8)
Coordinates:
  * target   (target) object 'GFP' 'RFP' 'Cy5' 'iRFP'
  * c        (c) int64 0 1 2 3
  * r        (r) int64 0

## Visualize the FOV

In [3]:
fov = experiment["fov_000"]

In [4]:
image = fov.get_image("primary")

In [5]:
import starfish
%gui qt
ipython = get_ipython()
ipython.magic("gui qt5")
starfish.display(image)

100%|██████████| 11/11 [00:00<00:00, 146.43it/s]


## Project onto the Z axis
The slider in the viewer should now only have as many options as there are channels. Also clipping the image to remove background noise.

In [5]:
import starfish
from starfish.types import Axes
clipper = starfish.image.Filter.Clip(p_min=99)
glp = starfish.image.Filter.GaussianLowPass(
    sigma=(1, 1, 1),
    is_volume=True
)
z_proj: starfish.ImageStack = ((image.max_proj(Axes.ZPLANE)))

100%|██████████| 44/44 [00:06<00:00,  7.27it/s]


In [6]:
starfish.display(z_proj)

100%|██████████| 1/1 [00:00<00:00, 121.78it/s]


## Apply Filters
Applying a bandpass and Gaussian low pass filters interleaved with clip filters.

In [6]:
from typing import Optional

from starfish import ImageStack
from starfish.image import Filter



def preprocess_fov(primary_fov_imagestack: ImageStack,
                  n_processes: Optional[int] = None,
                  is_volume: Optional[bool] = False) -> ImageStack:
   """Preprocess a Starfish field of view image stack in preparation for
   spot/pixel finding.

   NOTE: This preprocessing pipeline processes imagestacks in place!

   Args:
       primary_fov_imagestack (ImageStack): A starfish FOV Imagestack
       n_processes (Optional[int]): Number of processes to use for
           preprocessing steps. If None, uses the output of os.cpu_count().
           Defaults to None.

   Returns:
       ImageStack: A preprocessed starfish imagestack.
   """
   print("Applying First Clip...")
   first_clip = Filter.ClipPercentileToZero(p_min=75, p_max=100,
                                            is_volume=is_volume)
   first_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                  n_processes=n_processes)

   print("Applying Bandpass...")
   bpass = Filter.Bandpass(lshort=0.5, llong=7, threshold=1/(1<<16-1),
                           is_volume=is_volume)
   bpass.run(primary_fov_imagestack, in_place=True, verbose=True,
             n_processes=n_processes)

   print("Applying Second Clip...")
   second_clip = Filter.ClipValueToZero(v_min=1/(1<<16-1), is_volume=is_volume)
   second_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                   n_processes=n_processes)

   print("Applying Gaussian Low Pass...")
   z_gauss_filter = Filter.GaussianLowPass(sigma=(1, 0, 0), is_volume=True)
   z_gauss_filter.run(primary_fov_imagestack, in_place=True,
                      n_processes=n_processes)

   print("Applying Final Clips...")
   final_percent_clip = Filter.ClipPercentileToZero(p_min=90, min_coeff=1.75)
   final_percent_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                          n_processes=n_processes)

   final_value_clip = Filter.ClipValueToZero(v_max=1000/(1<<16-1))
   final_value_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                        n_processes=n_processes)

   return primary_fov_imagestack

In [7]:
z_proj = preprocess_fov(z_proj, n_processes=22)

100%|██████████| 4/4 [00:00<00:00, 28.31it/s]
0it [00:00, ?it/s]

Applying First Clip...


4it [00:00, 10.25it/s]
0it [00:00, ?it/s]

Applying Bandpass...


4it [00:00,  9.77it/s]
0it [00:00, ?it/s]

...

4it [00:00,  9.66it/s]


Applying Gaussian Low Pass...


0it [00:00, ?it/s]

Applying Final Clips...


4it [00:00, 10.27it/s]
4it [00:00, 10.52it/s]


Find spots
----------
Finally, a local blob detector that finds spots in each (z, y, x) volume
separately is applied. The user selects an "anchor round" and spots found in
all channels of that round are used to seed a local search across other rounds
and channels. The closest spot is selected, and any spots outside the search
radius (here 10 pixels) is discarded.

The Spot finder returns an IntensityTable containing all spots from round
zero. Note that many of the spots do _not_ identify spots in other rounds and
channels and will therefore fail decoding. Because of the stringency built
into the STARmap codebook, it is OK to be relatively permissive with the spot
finding parameters for this assay.

In [ ]:
import starfish
import time
import numpy as np
lsbd = starfish.spots.DetectSpots.BlobDetector(
     min_sigma=.45,
     max_sigma=1.25,
     num_sigma=25,
     threshold=np.percentile(np.ravel(z_proj.xarray.values), 97),
     is_volume=False,
     overlap=0.7,
# #     exclude_border=2,
# #     anchor_round=0,
# #     search_radius=10,
 )
# tlmpf = starfish.spots.DetectSpots.TrackpyLocalMaxPeakFinder(
#     spot_diameter=1,  # must be odd integer
#     min_mass=0.02,|
#     max_size=5,  # this is max radius
#     separation=1,
#     noise_size=0.65,  # this is not used because preprocess is False
#     preprocess=False,
#     percentile=10,  # this is irrelevant when min_mass, spot_diameter, and max_size are set properly
#     verbose=True,
#     is_volume=True,
# )
lmpf = starfish.spots.DetectSpots.LocalMaxPeakFinder(
min_distance=2,
stringency=0,
min_obj_area=4,
max_obj_area=600,
verbose=True,
is_volume=False)

# intensities = lsbd.run(z_proj, n_processes=22)
# intensities = tlmpf.run(image, n_processes=22)
start: float = time.perf_counter()
intensities = lmpf.run(z_proj, n_processes=22)
print(f"Minutes elapsed: {(start - time.perf_counter())/60}")

  0%|          | 0/100 [00:00<?, ?it/s]

Determining optimal threshold ...


  0%|          | 0/100 [00:00<?, ?it/s]

Determining optimal threshold ...


  0%|          | 0/100 [00:00<?, ?it/s]

...

Stopping early at threshold=0.00034101551687909347. Number of spots fell below: 3


 86%|████████▌ | 86/100 [02:54<00:27,  1.97s/it]

computing final spots ...
Stopping early at threshold=0.0024416753669466934. Number of spots fell below: 3


 88%|████████▊ | 88/100 [02:58<00:24,  2.00s/it]

computing final spots ...
Stopping early at threshold=0.015394293599658541. Number of spots fell below: 3
computing final spots ...


Decode spots
------------
Next, spots are decoded.

In [ ]:
decoded = experiment.codebook.decode_per_round_max(intensities.fillna(0))
decode_mask = decoded['target'] != 'nan'

In [35]:
import sklearn
print(sorted(sklearn.neighbors.VALID_METRICS['ball_tree']))
decoded = experiment.codebook.decode_metric(intensities.fillna(0), 1, 0, 2, 'l2')
decode_mask = decoded['target'] != 'nan'

['braycurtis', 'canberra', 'chebyshev', 'cityblock', 'dice', 'euclidean', 'hamming', 'haversine', 'infinity', 'jaccard', 'kulsinski', 'l1', 'l2', 'mahalanobis', 'manhattan', 'matching', 'minkowski', 'p', 'pyfunc', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'wminkowski']


In [ ]:
import pandas as pd
from starfish.types import Features
from starfish.core.types import DecodedSpots

df = pd.DataFrame(dict(decoded['features'].coords))
pixel_coordinates = pd.Index(['x', 'y', 'z'])
ds = DecodedSpots(df)

In [ ]:
ds.data

In [ ]:
%gui qt
viewer = starfish.display(
    image.max_proj(Axes.ZPLANE), decoded[decode_mask], radius_multiplier=2, mask_intensities=0.0
)

In [ ]:
counter={"GFP": 0, "RFP": 0, "Cy5": 0, "iRFP": 0}
for i,j in ds.data.iterrows():
    counter[j['target']]+=1
print(counter)